# Nupic Walkthrough

In this notebook we are testing:
1. Multi-Encoders: we are ecncoding multiple values and send them to our HTM-Model
1. Spatial Pooler (SP): analysing the shape and methods
1. Temporal Memory (TM): analysing the shape and methods
1. Anomaly Score: input, outputs
1. Feeforward Walk: the classic HTM-model
1. Walkback of the HTM-model: from TM back to the single input values

# Encoders

* Random Distributed Scalar Encoder
* Date/time

In [1]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv('../data/NAB/realKnownCause/machine_temperature_system_failure.csv')
data = data.to_dict(orient='records')
data[0:5]

[{'timestamp': '2013-12-02 21:15:00', 'value': 73.96732207},
 {'timestamp': '2013-12-02 21:20:00', 'value': 74.93588199999998},
 {'timestamp': '2013-12-02 21:25:00', 'value': 76.12416182},
 {'timestamp': '2013-12-02 21:30:00', 'value': 78.14070732},
 {'timestamp': '2013-12-02 21:35:00', 'value': 79.32983574}]

In [500]:
from nupic.encoders.random_distributed_scalar import RandomDistributedScalarEncoder
from nupic.encoders.scalar import ScalarEncoder

#RandomDistributedScalarEncoder?

In [501]:
# 21 bits with 3 active with buckets of size 5
#vEnc = RandomDistributedScalarEncoder(n=21, w=3, resolution=0.1)
vEnc = ScalarEncoder(n=128, w=21, minval=70, maxval=100, forced=True)
#vEnc = ScalarEncoder(resolution=0.1, w=21, minval=60, maxval=100)

print str(data[0]['value']) + " = ", vEnc.encode(data[0]['value'])
print str(data[1]['value']) + " = ", vEnc.encode(data[1]['value'])

73.96732207 =  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
74.935882 =  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


We will now encode to different values.

In [502]:
obs = []
inputVal1 = []
inputVal2 = []
inputSDR1 = []
inputSDR2 = []
inputSDR = []  # main
inputValIdx1 = [0, 127]
inputValIdx2 = [128, 255]

for i in xrange(5):
    obs.append(i)
    inputVal1.append(data[i]['value'])  # first i values
    inputVal2.append(data[-(i+1)]['value'])  # last -i values 
    inputSDR1.append(vEnc.encode(data[i]['value']))
    inputSDR2.append(vEnc.encode(data[-(i+1)]['value']))
    inputSDR.append(np.hstack((inputSDR1[i], inputSDR2[i])))  # combine the 2 ancoders in 1 enoder 
    
# send everything to dict    
track = pd.DataFrame({'inputVal1':inputVal1, 'inputVal2':inputVal2,
                      'inputSDR':inputSDR, 'inputSDR1':inputSDR1, 'inputSDR2':inputSDR2, 
                      #'inputValIdx1':inputValIdx1, 'inputVaIdx2':inputValIdx2
                     }, index=obs).to_dict(orient='records')
#track['index']

In [503]:
track

[{'inputSDR': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0], dtype=uint8),
  'inputSDR1': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
      

In [504]:
# import datetime
# from nupic.encoders.date import DateEncoder

# dateEnc = DateEncoder(season=5, weekend=1, dayOfWeek=7, timeOfDay=23) 

# tsObs1 = datetime.datetime.strptime(data[0]['timestamp'], "%Y-%m-%d %H:%M:%S")
# tsObs2 = datetime.datetime.strptime(data[1]['timestamp'], "%Y-%m-%d %H:%M:%S")

# print "TimeStamp-obs1 = ", dateEnc.encode(tsObs1)
# print "TimeStamp-obs2 = ", dateEnc.encode(tsObs2)
# len(dateEnc.encode(tsObs1))

# Spatial Pooler
[link to wiki](http://nupic.docs.numenta.org/1.0.3/api/algorithms/spatial-pooling.html#nupic.algorithms.spatial_pooler.SpatialPooler)

First we will check SP only on `Value`, without `Timestamp` 

In [505]:
from nupic.algorithms.spatial_pooler import SpatialPooler

In [506]:
print track[0]['inputSDR']
print len(track[0]['inputSDR'])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0]
256


In [507]:
# Define SP properties
sp = SpatialPooler(inputDimensions=(len(track[0]['inputSDR']),),
                   columnDimensions=(5,),
                   potentialRadius=30,
                   numActiveColumnsPerInhArea=1, # increasing this input, we got more activ SP columns when doinf sp.compute
                   globalInhibition=True,
                   synPermActiveInc=0,
                   #potentialPct=.80,
                   #stimulusThreshold=1, 
                   synPermConnected=.5,
                   seed=42)

In [508]:
#initialie SP
cols = []
connections = []

for col in xrange(sp.getColumnDimensions()):
    connected = np.zeros(len(track[0]['inputSDR']), dtype="int")
    sp.getConnectedSynapses(col, connected)
    cols.append(col)
    connections.append(connected)

spSDR = dict(zip(cols, connections))

print "SP form:"
spSDR.values()

SP form:


[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 1]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

The active bits (min-columns) are set by calculating the *overlapping score* with the input vector.  
*Overalpping score* = `inputSDR` * `spSDR\[column]`

In [509]:
# calculate ACTIVE columns: cols with higher overlap for every input SDR 
print "The active column for every `inputSDR` is:"

for i in xrange(len(track)):
    overlap = []
    for j in spSDR:
        overlap.append(sum(track[i]['inputSDR'] * spSDR.values()[j]))
    o = overlap.index(max(overlap))
    
    print "obs" + str(i) + ", overlap: ", str(overlap) +  ", Active col: ", str(o)

The active column for every `inputSDR` is:
obs0, overlap:  [6, 0, 0, 0, 7], Active col:  4
obs1, overlap:  [7, 0, 0, 0, 5], Active col:  0
obs2, overlap:  [8, 0, 0, 0, 7], Active col:  0
obs3, overlap:  [8, 0, 0, 0, 6], Active col:  0
obs4, overlap:  [6, 1, 0, 0, 5], Active col:  0


`compute()` should do the same.

In [510]:
for i in xrange(len(track)):
    output = np.zeros(sp.getColumnDimensions(), dtype="int")
    sp.compute(track[i]['inputSDR'], learn=True, activeArray=output)
    track[i]['sp_active'] = output.argmax() #save to dict
    
    print "obs" + str(i) + ", Active col: ", str(output)

obs0, Active col:  [0 0 0 0 1]
obs1, Active col:  [1 0 0 0 0]
obs2, Active col:  [1 0 0 0 0]
obs3, Active col:  [1 0 0 0 0]
obs4, Active col:  [1 0 0 0 0]


The `max()` in case of a tie, returns the first `maxVal` encountered.  
`sp.compute` apparently the last.

In [511]:
def last_max_index(lista):
    '''
    max() function in case of a tie returns the first max value found.
    This function, in case of tie returns the last max value in our list.
    
    input: list
    output: int, index last max value 
    '''
    
    l = len(lista)
    lista.reverse()
    i = lista.index(max(lista))
    lista.reverse()
    return (l-i-1)

In [512]:
# calculate ACTIVE columns: cols with higher overlap for every input SDR 
for i in xrange(len(track)):
    overlap = []
    for j in spSDR:
        overlap.append(sum(track[i]['inputSDR'] * spSDR.values()[j]))
    o = last_max_index(overlap)
        
    print "obs" + str(i) + ", overlap: ", str(overlap) +  ", Active col: ", str(o)

obs0, overlap:  [6, 0, 0, 0, 7], Active col:  4
obs1, overlap:  [7, 0, 0, 0, 5], Active col:  0
obs2, overlap:  [8, 0, 0, 0, 7], Active col:  0
obs3, overlap:  [8, 0, 0, 0, 6], Active col:  0
obs4, overlap:  [6, 1, 0, 0, 5], Active col:  0


Now the values matches with `sp.compute`.

In [513]:
for _ in xrange(1000):
    sp.compute(track[1]['inputSDR'], learn=True, activeArray=output)

In [514]:
print "ACTIVE Column: ", output

ACTIVE Column:  [1 0 0 0 0]


Since `learn=False`, the synapses will not be updated, so that the output never chance.
Furthermore we set `SpatialPooler(synPermActiveInc=0, synPermInactiveDec=0)`, so that even with `learn=True` the SP is not able to learn. 

In [515]:
# Permanence
permanence = []

for i in xrange(sp.getColumnDimensions()):
    p = []
    sp.getPermanence(i, p)
    permanence.append(np.array(p))

# permanence

For `Permanence > Threshold` we have a connection to the *inputSDR*.

Summarizing SP:

In [516]:
print "inputSDR: ", track[0]['inputSDR']
print "SP active col: ", spSDR[track[0]['sp_active']]
print "Overlappin bits: ", str(track[0]['inputSDR'] * spSDR[track[0]['sp_active']])
print "Pemanence winning col: ", permanence[track[0]['sp_active']]

inputSDR:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0]
SP active col:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0
 1 

# Temporal Pooler

[link to wiki](http://nupic.docs.numenta.org/1.0.3/api/algorithms/sequence-memory.html#nupic.algorithms.backtracking_tm_cpp.BacktrackingTMCPP)
SP output is TM input.

In [517]:
from nupic.algorithms.backtracking_tm import BacktrackingTM

In [518]:
spSDR[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1])

To better understand the example we suggest to set `verbosity=5`

In [521]:
# define TM properties
# input vectors to feed TM: SP output is TM input and must be numberOfCols wide. 
tm = BacktrackingTM(numberOfCols=len(spSDR[0]), cellsPerColumn=3,
                    initialPerm=0.5, connectedPerm=0.5,
                    minThreshold=10, newSynapseCount=10,
                    permanenceInc=0.1, permanenceDec=0.0,
                    activationThreshold=2,
                    globalDecay=0, burnIn=1,
                    checkSynapseConsistency=False,
                    pamLength=10,
                    seed = 42,
                    collectStats=True,
                    #verbosity=5,
                    )

In [530]:
tm0= tm.compute(spSDR[0], enableInference=True, enableLearn=True)
print tm0.shape
print tm0.reshape(tm.numberOfCols, tm.cellsPerColumn)

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 1.,  1.,  1.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 1.,  1.,  1.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 1.,  1.,  1.],
       [ 0.,  0.,  0.],
       [ 1.,  1.,  1.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 1.,  1.,  1.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.

In [531]:
tm.getStats()

{'curExtra': 0,
 'curFalseNegativeScore': 1.0,
 'curFalsePositiveScore': 0.0,
 'curMissing': 13,
 'curPredictionScore2': 0.0,
 'falseNegativeAvg': 1.0,
 'falsePositiveAvg': 0.0,
 'nPredictions': 1,
 'pctExtraAvg': 0.0,
 'pctMissingAvg': 100.0,
 'predictionScoreAvg2': 0.0,
 'prevSequenceSignature': None,
 'totalExtra': 0,
 'totalMissing': 13}

**Example1**
`enableInference=True`, `enableLearn=False`

In [532]:
tm4 = tm.compute(spSDR[4], enableInference=True, enableLearn=False)
tm4.reshape(tm.numberOfCols, tm.cellsPerColumn)

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.

**Example 2**
`enableInference=False`, `enableLearn=True`

In [315]:
tm4 = tm.compute(spSDR[4], enableInference=False, enableLearn=True)
tm4.reshape(tm.numberOfCols, tm.cellsPerColumn)

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.

**Example 3** `enableInference=False`, `enableLearn=True`

In [316]:
tm4 = tm.compute(spSDR[4], enableInference=True, enableLearn=True)
tm4.reshape(tm.numberOfCols, tm.cellsPerColumn)

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.

In [576]:
x = spSDR
#x.values()

In [577]:
# Step 3: send the  input to the temporal memory for learning
# We repeat the sequence 10 times
for i in range(10):

    # Send each input in the sequence in order
    for j in range(len(x)):

        # The compute method performs one step of learning and/or inference. Note:
        # here we just perform learning but you can perform prediction/inference and
        # learning in the same step if you want (online learning).
        tm_output = tm.compute(x[j], enableLearn=True, enableInference=True)
        tm_output.reshape(tm.numberOfCols, tm.cellsPerColumn)
        # This function prints the segments associated with every cell.$$$$
        # If you really want to understand the TP, uncomment this line. By following
        # every step you can get an excellent understanding for exactly how the TP
        # learns.
        #tm.printCells()

    # The reset command tells the TM that a sequence just ended and essentially
    # zeros out all the states. It is not strictly necessary but it's a bit
    # messier without resets, and the TM learns quicker with resets.
    tm.reset

**..also proof Anomaly Score** (TODO)

In [578]:
from nupic.algorithms.anomaly import computeRawAnomalyScore

In [580]:
# Step 4: send the same sequence of vectors and look at predictions made by
# temporal memory and check Anomaly Score

# Utility routine for printing the input vector
def formatRow(x):
    s = ''
    for c in range(len(x)):
        if c > 0 and c % 10 == 0:
            s += ' '
        s += str(x[c])
    s += ' '
    return s

for j in range(5):
    print "\n\n--------","01234"[j],"-----------"
    print "Raw input vector\n",formatRow(x[j])

    # Send each vector to the TP, with learning turned off
    tm.compute(x[j], enableLearn=True, enableInference=True)

    # This method prints out the active state of each cell followed by the
    # predicted state of each cell. For convenience the cells are grouped
    # 10 at a time. When there are multiple cells per column the printout
    # is arranged so the cells in a column are stacked together
    #
    # What you should notice is that the columns where active state is 1
    # represent the SDR for the current input pattern and the columns where
    # predicted state is 1 represent the SDR for the next expected pattern
    print "\nAll the active and predicted cells:"
    tm.printStates(printPrevious=False, printLearnState=False)

    # tm.getPredictedState() gets the predicted cells.
    # predictedCells[c][i] represents the state of the i'th cell in the c'th
    # column. To see if a column is predicted, we can simply take the OR
    # across all the cells in that column. In numpy we can do this by taking
    # the max along axis 1.
    print "\n\nThe following columns are predicted by the temporal memory. This"
    print "should correspond to columns in the *next* item in the sequence."
    predictedCells = tm.getPredictedState()
    print formatRow(predictedCells.max(axis=1).nonzero())
    print "Anomaly Score: ", computeRawAnomalyScore(idx_active_cols(tm.infActiveState['t']), idx_active_cols(tm.cellConfidence['t-1']))



-------- 0 -----------
Raw input vector
0000000000 1000010000 0000000101 0011110010 0010100000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 000101 

All the active and predicted cells:

Inference Active state
0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 000000 
0000000000 1000000000 0000000100 0000010000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 000100 
0000000000 0000010000 0000000001 0011100010 0010

In [379]:
#predictedCells.T
tm.getPredictedState().T

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [381]:
#tm.cellConfidence

# Anomaly Score

[link](http://nupic.docs.numenta.org/stable/guides/anomaly-detection.html)

The algorithm for the anomaly score is as follows:  

AS = |A_(t) - (P_(t-1) cross A_(t))|  / |A_(t)|  

A_(t):   Predicted columns at time t  
P_(t-1): Active columns at time t

**Note**: Here, a “predicted column” is a column with a non-zero confidence value. This is not exactly the same as having a cell in the predicted state. For more information, refer the “predicted cells vs. confidences” section below.  

...to compute the confidences for a cell, the Temporal Pooler uses the soft match count (the number of active synapses, regardless of the permanence values). Therefore, the set of columns with non-zero confidences will always be a superset of the columns containing predicted cells.

In [581]:
def computeRawAnomalyScore(activeColumns, prevPredictedColumns):
  """Computes the raw anomaly score.

  The raw anomaly score is the fraction of active columns not predicted.

  :param activeColumns: array of active column indices
  :param prevPredictedColumns: array of columns indices predicted in prev step
  :returns: anomcaly score 0..1 (float)
  """
  nActiveColumns = len(activeColumns)
  if nActiveColumns > 0:
    # Test whether each element of a 1-D array is also present in a second
    # array. Sum to get the total # of columns that are active and were
    # predicted.
    score = numpy.in1d(activeColumns, prevPredictedColumns).sum()
    # Get the percent of active columns that were NOT predicted, that is
    # our anomaly score.
    score = (nActiveColumns - score) / float(nActiveColumns)
  else:
    # There are no active columns.
    score = 0.0

  return score

Expolring the two arguments that the function takes in:

In [582]:
formatRow(tm.cellConfidence['t-1'].max(axis=1).nonzero())  # forecast in 't-1' for 't'

'[200 204 206 211 219 222 225 227 229 235 238 239 242] '

In [583]:
formatRow(tm.infPredictedState['t-1'].max(axis=1).nonzero())

'[200 204 206 211 219 222 225 227 229 235 238 239 242] '

`tm.cellConfidence['t-1']` is the same as `tm.infPredictedState['t-1']`:  
The only difference is that instead of 1s, `cellConfidence` has values >0, <1.  
Eventually `tm.cellConfidence['t-1']` should contain even more possible outcomes.  
`inPredictedState[t]` predicted to be active next,  
`inPredictedState[t-1]` predicted before, for current input 

In [592]:
np.flatnonzero(np.array(tm.infActiveState['t'])) # index of active cells in in  't' 

array([602, 613, 619, 635, 659, 668, 677, 682, 689, 706, 716, 719, 728])

ISSUE: `np.flatnonzero` first flats he array and then returns the idx

In [593]:
def idx_active_cols(inputArr):
    """
    This function takes an array and returns an array with the index for ACTIVE columns: 
    
    Input:   a 1D or nD arraz
    Output:  arr with index of active cols
    """
    #tmObject.reshape(tmObject.numberOfCols, tm.cellsPerColumn)
    activeColsVec = []  # initialize vector

    for i in range(inputArr.shape[0]):
        # assign 1 if any 1 (active cell) in the column,
        # 0 otherwise
        if np.any(inputArr[i]>0):
        # if np.any(tm.compute(spSDR[track[3]['sp_active']], enableLearn=True, enableInference=True).reshape(256, 3)[i]>0):
            activeColsVec.append(1)
        else:
            activeColsVec.append(0)
    # return index of active Columns        
    tmActiveColsIdx = np.flatnonzero(np.array(activeColsVec))
    return tmActiveColsIdx

In [594]:
idx_active_cols(tm.infActiveState['t'])

array([200, 204, 206, 211, 219, 222, 225, 227, 229, 235, 238, 239, 242])

In [595]:
# np.flatnonzero returns index nonzero entries
#computeRawAnomalyScore(np.flatnonzero(np.array(tm.infActiveState['t'])), np.flatnonzero(np.array(tm.cellConfidence['t-1'])))

computeRawAnomalyScore(idx_active_cols(tm.infActiveState['t']), idx_active_cols(tm.cellConfidence['t-1']))


0.0

Some proof:

In [597]:
computeRawAnomalyScore(idx_active_cols(tm.infActiveState['t']), idx_active_cols(tm.cellConfidence['t-1']))
print idx_active_cols(tm.infActiveState['t'])
print idx_active_cols(tm.cellConfidence['t-1'])
print idx_active_cols(tm.infPredictedState['t-1'])
# tm.infPredictedState # [t] predicted to be active next, [t-1] predicted before, for current input 

[200 204 206 211 219 222 225 227 229 235 238 239 242]
[200 204 206 211 219 222 225 227 229 235 238 239 242]
[200 204 206 211 219 222 225 227 229 235 238 239 242]


More outputs has been predicted, one of the them occured.

# Trackability - Feedforward

In [382]:
track[0]

{'inputSDR': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0], dtype=uint8),
 'inputSDR1': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1,

In [383]:
print track[0]['inputVal1']
print track[0]['inputVal2']

73.96732207
96.90386085


**Decoding** using the main `inputSDR` it decodes only the first 128bits, the length it was built for

In [384]:
vEnc.decode(track[0]['inputSDR'])

({'[70:100]': ([[73.925233644859816, 73.925233644859816]], '73.93')},
 ['[70:100]'])

In [385]:
vEnc.decode(track[0]['inputSDR1'])

({'[70:100]': ([[73.925233644859816, 73.925233644859816]], '73.93')},
 ['[70:100]'])

In [386]:
vEnc.decode(track[0]['inputSDR2'])

({'[70:100]': ([[96.915887850467286, 96.915887850467286]], '96.92')},
 ['[70:100]'])

# Trackability - Backwards

Generate *TM output* by feeding in the active columns of the *SP*.

In [624]:
track[0]['sp_active'] 

4

In [625]:
spSDR[track[0]['sp_active']]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0])

In [629]:
tm.compute(spSDR[track[0]['sp_active']], enableLearn=True, enableInference=True).reshape(tm.numberOfCols, tm.cellsPerColumn)

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.

Calculate Anomaly Score:

In [630]:
computeRawAnomalyScore(idx_active_cols(tm.infActiveState['t']), idx_active_cols(tm.cellConfidence['t-1']))

0.0

Since `AS=0`, we can cause it to be higher changing some value: 

In [639]:
# Activate 5 random columns within the first 200 
tmInfActiveStateTemp = np.copy(tm.infActiveState['t'])
tmInfActiveStateTemp[np.random.randint(0, 200, size=5)] = 1
idx_active_cols(tmInfActiveStateTemp)

array([ 77, 104, 112, 153, 156, 200, 204, 206, 211, 219, 222, 225, 227,
       229, 235, 238, 239, 242])

In [643]:
computeRawAnomalyScore(idx_active_cols(tmInfActiveStateTemp), idx_active_cols(tm.cellConfidence['t-1']))

0.27777777777777779

Once an AS has been outputed, we want to know which are the cells the where unpredicted and causes the error to raise.
Which are the columns that where not correclty predicted?

In [644]:
idxAS = np.in1d(idx_active_cols(tmInfActiveStateTemp), idx_active_cols(tm.cellConfidence['t-1']))

In [646]:
idx_active_cols(tmInfActiveStateTemp)[idxAS == False]

array([ 77, 104, 112, 153, 156])

From here on we can execute the all the steps backward, since we know which decoder is responsible for each range of bits.

Continuing with an existing sp..

Build `spSDR[track[3]['sp_active']]` back:

In [648]:
sp_active = np.zeros_like(spSDR[0])
sp_active[idx_active_cols(tm.infActiveState['t'])] = 1
sp_active

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0])

Find the corresponding column in spSDR:

In [649]:
idx = []
for _ in spSDR:
    i =+ 1
    if np.array_equal(sp_active, spSDR[_]) == True:
        idx.append(i)
print "matching spSDR:", idx

matching spSDR: [1]


In [650]:
# calculate with which inputSDR, the active SP col has the higher overlap: 

for j in idx:
    overlap = []
    for i in xrange(len(track)):
        overlap.append(sum(track[i]['inputSDR'] * spSDR[j]))
        o = last_max_index(overlap)
        
    print "overlap: ", str(overlap) +  "\nInputSDR[idx]: ", str(o)
    print "inputSDR: ", str(track[o]['inputSDR'])

overlap:  [0, 0, 0, 0, 1]
InputSDR[idx]:  4
inputSDR:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0]


This function should be useful in case we have ties in the overlap-score.

In [651]:
def match_back_SP_to_SDR(lista):
    '''
    This fnc returns the indeces of the InputSDR/s that match 
    (have the highest overalpping score) the current SP the most.
    
    input:  copy of a list `list[:]` with the overlap score bw. 
            the winning spSDR[i] and the inputSDR[0:]   
    output: 'match', a list, indeces of InputSDR in `track`
    '''
    
    a = max(lista)
    b = a
    match = []
    count = 0

    while b == a:
        i = lista.index(b)
        out = lista.pop(i)
        i = i+count  # fill the indexes popped out
        match.append(i)
        count += 1
        b = max(lista)    
    
    return match

In [652]:
match_back_SP_to_SDR(overlap[:])

[4]

In [653]:
for i in match_back_SP_to_SDR(overlap[:]):
    print "inputVal1[" + str(i) + "]: " + str(track[i]['inputVal1'])
    print "De-Encoder1: " + str(vEnc.decode(track[i]['inputSDR'][inputValIdx1[0]:inputValIdx1[1]]))
    print "inputVal2[" + str(i) + "]: " + str(track[i]['inputVal2'])
    print "De-Encoder2: " + str(vEnc.decode(track[i]['inputSDR'][inputValIdx2[0]:inputValIdx2[1]]))
    print "-------"

inputVal1[4]: 79.32983574
De-Encoder1: ({'[70:100]': ([[79.252336448598129, 79.252336448598129]], '79.25')}, ['[70:100]'])
inputVal2[4]: 98.18541493
De-Encoder2: ({'[70:100]': ([[98.317757009345797, 98.317757009345797]], '98.32')}, ['[70:100]'])
-------


In [654]:
track[0]

{'inputSDR': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0], dtype=uint8),
 'inputSDR1': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1,

# Model Parameters

`MODEL_PARAMS` have all of the parameters for the CLA model and subcomponents

In [290]:
# Model Params!
MODEL_PARAMS = {
    # Type of model that the rest of these parameters apply to.
    'model': "HTMPrediction",

    # Version that specifies the format of the config.
    'version': 1,

    # Intermediate variables used to compute fields in modelParams and also
    # referenced from the control section.
    'aggregationInfo': {   'days': 0,
        'fields': [('consumption', 'sum')],
        'hours': 1,
        'microseconds': 0,
        'milliseconds': 0,
        'minutes': 0,
        'months': 0,
        'seconds': 0,
        'weeks': 0,
        'years': 0},

    'predictAheadTime': None,

    # Model parameter dictionary.
    'modelParams': {
        # The type of inference that this model will perform
        'inferenceType': 'TemporalMultiStep',

        'sensorParams': {
            # Sensor diagnostic output verbosity control;
            # if > 0: sensor region will print out on screen what it's sensing
            # at each step 0: silent; >=1: some info; >=2: more info;
            # >=3: even more info (see compute() in py/regions/RecordSensor.py)
            'verbosity' : 0,

            # Include the encoders we use
            'encoders': {
                u'timestamp_timeOfDay': {
                    'fieldname': u'timestamp',
                    'name': u'timestamp_timeOfDay',
                    'timeOfDay': (21, 0.5),
                    'type': 'DateEncoder'
                },
                u'timestamp_dayOfWeek': None,
                u'timestamp_weekend': None,
                u'consumption': {
                    'clipInput': True,
                    'fieldname': u'consumption',
                    'maxval': 100.0,
                    'minval': 0.0,
                    'n': 50,
                    'name': u'c1',
                    'type': 'ScalarEncoder',
                    'w': 21
                },
            },

            # A dictionary specifying the period for automatically-generated
            # resets from a RecordSensor;
            #
            # None = disable automatically-generated resets (also disabled if
            # all of the specified values evaluate to 0).
            # Valid keys is the desired combination of the following:
            #   days, hours, minutes, seconds, milliseconds, microseconds, weeks
            #
            # Example for 1.5 days: sensorAutoReset = dict(days=1,hours=12),
            #
            # (value generated from SENSOR_AUTO_RESET)
            'sensorAutoReset' : None,
        },

        'spEnable': True,

        'spParams': {
            # SP diagnostic output verbosity control;
            # 0: silent; >=1: some info; >=2: more info;
            'spVerbosity' : 0,

            # Spatial Pooler implementation selector, see getSPClass
            # in py/regions/SPRegion.py for details
            # 'py' (default), 'cpp' (speed optimized, new)
            'spatialImp' : 'cpp',

            'globalInhibition': 1,

            # Number of cell columns in the cortical region (same number for
            # SP and TM)
            # (see also tpNCellsPerCol)
            'columnCount': 2048,

            'inputWidth': 0,

            # SP inhibition control (absolute value);
            # Maximum number of active columns in the SP region's output (when
            # there are more, the weaker ones are suppressed)
            'numActiveColumnsPerInhArea': 40,

            'seed': 1956,

            # potentialPct
            # What percent of the columns's receptive field is available
            # for potential synapses. At initialization time, we will
            # choose potentialPct * (2*potentialRadius+1)^2
            'potentialPct': 0.5,

            # The default connected threshold. Any synapse whose
            # permanence value is above the connected threshold is
            # a "connected synapse", meaning it can contribute to the
            # cell's firing. Typical value is 0.10. Cells whose activity
            # level before inhibition falls below minDutyCycleBeforeInh
            # will have their own internal synPermConnectedCell
            # threshold set below this default value.
            # (This concept applies to both SP and TM and so 'cells'
            # is correct here as opposed to 'columns')
            'synPermConnected': 0.1,

            'synPermActiveInc': 0.1,

            'synPermInactiveDec': 0.005,
        },

        # Controls whether TM is enabled or disabled;
        # TM is necessary for making temporal predictions, such as predicting
        # the next inputs.  Without TP, the model is only capable of
        # reconstructing missing sensor inputs (via SP).
        'tmEnable' : True,

        'tmParams': {
            # TM diagnostic output verbosity control;
            # 0: silent; [1..6]: increasing levels of verbosity
            # (see verbosity in nupic/trunk/py/nupic/research/TP.py and BacktrackingTMCPP.py)
            'verbosity': 0,

            # Number of cell columns in the cortical region (same number for
            # SP and TM)
            # (see also tpNCellsPerCol)
            'columnCount': 2048,

            # The number of cells (i.e., states), allocated per column.
            'cellsPerColumn': 32,

            'inputWidth': 2048,

            'seed': 1960,

            # Temporal Pooler implementation selector (see _getTPClass in
            # CLARegion.py).
            'temporalImp': 'cpp',

            # New Synapse formation count
            # NOTE: If None, use spNumActivePerInhArea
            #
            # TODO: need better explanation
            'newSynapseCount': 20,

            # Maximum number of synapses per segment
            #  > 0 for fixed-size CLA
            # -1 for non-fixed-size CLA
            #
            # TODO: for Ron: once the appropriate value is placed in TP
            # constructor, see if we should eliminate this parameter from
            # description.py.
            'maxSynapsesPerSegment': 32,

            # Maximum number of segments per cell
            #  > 0 for fixed-size CLA
            # -1 for non-fixed-size CLA
            #
            # TODO: for Ron: once the appropriate value is placed in TP
            # constructor, see if we should eliminate this parameter from
            # description.py.
            'maxSegmentsPerCell': 128,

            # Initial Permanence
            # TODO: need better explanation
            'initialPerm': 0.21,

            # Permanence Increment
            'permanenceInc': 0.1,

            # Permanence Decrement
            # If set to None, will automatically default to tpPermanenceInc
            # value.
            'permanenceDec' : 0.1,

            'globalDecay': 0.0,

            'maxAge': 0,

            # Minimum number of active synapses for a segment to be considered
            # during search for the best-matching segments.
            # None=use default
            # Replaces: tpMinThreshold
            'minThreshold': 9,

            # Segment activation threshold.
            # A segment is active if it has >= tpSegmentActivationThreshold
            # connected synapses that are active due to infActiveState
            # None=use default
            # Replaces: tpActivationThreshold
            'activationThreshold': 12,

            'outputType': 'normal',

            # "Pay Attention Mode" length. This tells the TM how many new
            # elements to append to the end of a learned sequence at a time.
            # Smaller values are better for datasets with short sequences,
            # higher values are better for datasets with long sequences.
            'pamLength': 1,
        },

        'clParams': {
            'regionName' : 'SDRClassifierRegion',

            # Classifier diagnostic output verbosity control;
            # 0: silent; [1..6]: increasing levels of verbosity
            'verbosity' : 0,

            # This controls how fast the classifier learns/forgets. Higher values
            # make it adapt faster and forget older patterns faster.
            'alpha': 0.005,

            # This is set after the call to updateConfigFromSubConfig and is
            # computed from the aggregationInfo and predictAheadTime.
            'steps': '1,5',

            'implementation': 'cpp',
        },

        'trainSPNetOnlyIfRequested': False,
    },
}

# Dataset Helpers

In [38]:
from pkg_resources import resource_filename

datasetPath = resource_filename("nupic.datafiles", "extra/hotgym/hotgym.csv")
print datasetPath

with open(datasetPath) as inputFile:
    print
    for _ in xrange(8):
        print inputFile.next().strip()

/Users/mleborgne/_git/nupic/src/nupic/datafiles/extra/hotgym/hotgym.csv

gym,address,timestamp,consumption
string,string,datetime,float
S,,T,
Balgowlah Platinum,Shop 67 197-215 Condamine Street Balgowlah 2093,2010-07-02 00:00:00.0,5.3
Balgowlah Platinum,Shop 67 197-215 Condamine Street Balgowlah 2093,2010-07-02 00:15:00.0,5.5
Balgowlah Platinum,Shop 67 197-215 Condamine Street Balgowlah 2093,2010-07-02 00:30:00.0,5.1
Balgowlah Platinum,Shop 67 197-215 Condamine Street Balgowlah 2093,2010-07-02 00:45:00.0,5.3
Balgowlah Platinum,Shop 67 197-215 Condamine Street Balgowlah 2093,2010-07-02 01:00:00.0,5.2


# Loading Data

`FileRecordStream` - file reader for the NuPIC file format (CSV with three header rows, understands datetimes)

In [39]:
from nupic.data.file_record_stream import FileRecordStream

def getData():
    return FileRecordStream(datasetPath)

data = getData()
for _ in xrange(5):
    print data.next()

['Balgowlah Platinum', 'Shop 67 197-215 Condamine Street Balgowlah 2093', datetime.datetime(2010, 7, 2, 0, 0), 5.3]
['Balgowlah Platinum', 'Shop 67 197-215 Condamine Street Balgowlah 2093', datetime.datetime(2010, 7, 2, 0, 15), 5.5]
['Balgowlah Platinum', 'Shop 67 197-215 Condamine Street Balgowlah 2093', datetime.datetime(2010, 7, 2, 0, 30), 5.1]
['Balgowlah Platinum', 'Shop 67 197-215 Condamine Street Balgowlah 2093', datetime.datetime(2010, 7, 2, 0, 45), 5.3]
['Balgowlah Platinum', 'Shop 67 197-215 Condamine Street Balgowlah 2093', datetime.datetime(2010, 7, 2, 1, 0), 5.2]


In [40]:
from nupic.frameworks.opf.model_factory import ModelFactory
model = ModelFactory.create(MODEL_PARAMS)
model.enableInference({'predictedField': 'consumption'})

In [41]:
data = getData()
for _ in xrange(100):
    record = dict(zip(data.getFieldNames(), data.next()))
    print "input: ", record["consumption"]
    result = model.run(record)
    print "prediction: ", result.inferences["multiStepBestPredictions"][1]

input:  5.3
prediction:  5.3
input:  5.5
prediction:  5.5
input:  5.1
prediction:  5.36
input:  5.3
prediction:  5.1
input:  5.2
prediction:  5.342
input:  5.5
prediction:  5.2994
input:  4.5
prediction:  5.35958
input:  1.2
prediction:  4.92
input:  1.1
prediction:  1.2
input:  1.2
prediction:  1.17
input:  1.2
prediction:  1.179
input:  1.2
prediction:  1.1853
input:  1.2
prediction:  1.18971
input:  1.2
prediction:  1.192797
input:  1.1
prediction:  1.1949579
input:  1.2
prediction:  1.16647053
input:  1.1
prediction:  1.176529371
input:  1.2
prediction:  1.1535705597
input:  1.2
prediction:  1.16749939179
input:  1.1
prediction:  1.17724957425
input:  1.2
prediction:  1.15407470198
input:  6.0
prediction:  1.16785229138
input:  7.9
prediction:  5.551706
input:  8.4
prediction:  6.2561942
input:  10.6
prediction:  6.89933594
input:  12.4
prediction:  10.6
input:  12.1
prediction:  12.4
input:  12.4
prediction:  12.31
input:  11.4
prediction:  12.337
input:  11.2
prediction:  10.84
i

In [42]:
print "5-step prediction: ", result.inferences["multiStepBestPredictions"][5]

5-step prediction:  1.19932370691


# Anomaly Score

In [43]:
# Model Params!
MODEL_PARAMS = {
    # Type of model that the rest of these parameters apply to.
    'model': "HTMPrediction",

    # Version that specifies the format of the config.
    'version': 1,

    # Intermediate variables used to compute fields in modelParams and also
    # referenced from the control section.
    'aggregationInfo': {   'days': 0,
        'fields': [('consumption', 'sum')],
        'hours': 1,
        'microseconds': 0,
        'milliseconds': 0,
        'minutes': 0,
        'months': 0,
        'seconds': 0,
        'weeks': 0,
        'years': 0},

    'predictAheadTime': None,

    # Model parameter dictionary.
    'modelParams': {
        # The type of inference that this model will perform
        'inferenceType': 'TemporalAnomaly',

        'sensorParams': {
            # Sensor diagnostic output verbosity control;
            # if > 0: sensor region will print out on screen what it's sensing
            # at each step 0: silent; >=1: some info; >=2: more info;
            # >=3: even more info (see compute() in py/regions/RecordSensor.py)
            'verbosity' : 0,

            # Include the encoders we use
            'encoders': {
                u'timestamp_timeOfDay': {
                    'fieldname': u'timestamp',
                    'name': u'timestamp_timeOfDay',
                    'timeOfDay': (21, 0.5),
                    'type': 'DateEncoder'},
                u'timestamp_dayOfWeek': None,
                u'timestamp_weekend': None,
                u'consumption': {
                    'clipInput': True,
                    'fieldname': u'consumption',
                    'maxval': 100.0,
                    'minval': 0.0,
                    'n': 50,
                    'name': u'c1',
                    'type': 'ScalarEncoder',
                    'w': 21},},

            # A dictionary specifying the period for automatically-generated
            # resets from a RecordSensor;
            #
            # None = disable automatically-generated resets (also disabled if
            # all of the specified values evaluate to 0).
            # Valid keys is the desired combination of the following:
            #   days, hours, minutes, seconds, milliseconds, microseconds, weeks
            #
            # Example for 1.5 days: sensorAutoReset = dict(days=1,hours=12),
            #
            # (value generated from SENSOR_AUTO_RESET)
            'sensorAutoReset' : None,
        },

        'spEnable': True,

        'spParams': {
            # SP diagnostic output verbosity control;
            # 0: silent; >=1: some info; >=2: more info;
            'spVerbosity' : 0,

            # Spatial Pooler implementation selector, see getSPClass
            # in py/regions/SPRegion.py for details
            # 'py' (default), 'cpp' (speed optimized, new)
            'spatialImp' : 'cpp',

            'globalInhibition': 1,

            # Number of cell columns in the cortical region (same number for
            # SP and TM)
            # (see also tpNCellsPerCol)
            'columnCount': 2048,

            'inputWidth': 0,

            # SP inhibition control (absolute value);
            # Maximum number of active columns in the SP region's output (when
            # there are more, the weaker ones are suppressed)
            'numActiveColumnsPerInhArea': 40,

            'seed': 1956,

            # potentialPct
            # What percent of the columns's receptive field is available
            # for potential synapses. At initialization time, we will
            # choose potentialPct * (2*potentialRadius+1)^2
            'potentialPct': 0.5,

            # The default connected threshold. Any synapse whose
            # permanence value is above the connected threshold is
            # a "connected synapse", meaning it can contribute to the
            # cell's firing. Typical value is 0.10. Cells whose activity
            # level before inhibition falls below minDutyCycleBeforeInh
            # will have their own internal synPermConnectedCell
            # threshold set below this default value.
            # (This concept applies to both SP and TM and so 'cells'
            # is correct here as opposed to 'columns')
            'synPermConnected': 0.1,

            'synPermActiveInc': 0.1,

            'synPermInactiveDec': 0.005,
        },

        # Controls whether TM is enabled or disabled;
        # TM is necessary for making temporal predictions, such as predicting
        # the next inputs.  Without TP, the model is only capable of
        # reconstructing missing sensor inputs (via SP).
        'tmEnable' : True,

        'tmParams': {
            # TM diagnostic output verbosity control;
            # 0: silent; [1..6]: increasing levels of verbosity
            # (see verbosity in nupic/trunk/py/nupic/research/TP.py and BacktrackingTMCPP.py)
            'verbosity': 0,

            # Number of cell columns in the cortical region (same number for
            # SP and TM)
            # (see also tpNCellsPerCol)
            'columnCount': 2048,

            # The number of cells (i.e., states), allocated per column.
            'cellsPerColumn': 32,

            'inputWidth': 2048,

            'seed': 1960,

            # Temporal Pooler implementation selector (see _getTPClass in
            # CLARegion.py).
            'temporalImp': 'cpp',

            # New Synapse formation count
            # NOTE: If None, use spNumActivePerInhArea
            #
            # TODO: need better explanation
            'newSynapseCount': 20,

            # Maximum number of synapses per segment
            #  > 0 for fixed-size CLA
            # -1 for non-fixed-size CLA
            #
            # TODO: for Ron: once the appropriate value is placed in TP
            # constructor, see if we should eliminate this parameter from
            # description.py.
            'maxSynapsesPerSegment': 32,

            # Maximum number of segments per cell
            #  > 0 for fixed-size CLA
            # -1 for non-fixed-size CLA
            #
            # TODO: for Ron: once the appropriate value is placed in TP
            # constructor, see if we should eliminate this parameter from
            # description.py.
            'maxSegmentsPerCell': 128,

            # Initial Permanence
            # TODO: need better explanation
            'initialPerm': 0.21,

            # Permanence Increment
            'permanenceInc': 0.1,

            # Permanence Decrement
            # If set to None, will automatically default to tpPermanenceInc
            # value.
            'permanenceDec' : 0.1,

            'globalDecay': 0.0,

            'maxAge': 0,

            # Minimum number of active synapses for a segment to be considered
            # during search for the best-matching segments.
            # None=use default
            # Replaces: tpMinThreshold
            'minThreshold': 9,

            # Segment activation threshold.
            # A segment is active if it has >= tpSegmentActivationThreshold
            # connected synapses that are active due to infActiveState
            # None=use default
            # Replaces: tpActivationThreshold
            'activationThreshold': 12,

            'outputType': 'normal',

            # "Pay Attention Mode" length. This tells the TM how many new
            # elements to append to the end of a learned sequence at a time.
            # Smaller values are better for datasets with short sequences,
            # higher values are better for datasets with long sequences.
            'pamLength': 1,
        },

        'clParams': {
            'regionName' : 'SDRClassifierRegion',

            # Classifier diagnostic output verbosity control;
            # 0: silent; [1..6]: increasing levels of verbosity
            'verbosity' : 0,

            # This controls how fast the classifier learns/forgets. Higher values
            # make it adapt faster and forget older patterns faster.
            'alpha': 0.005,

            # This is set after the call to updateConfigFromSubConfig and is
            # computed from the aggregationInfo and predictAheadTime.
            'steps': '1',

            'implementation': 'cpp',
        },

        'anomalyParams': {
            u'anomalyCacheRecords': None,
            u'autoDetectThreshold': None,
            u'autoDetectWaitRecords': 2184
        },

        'trainSPNetOnlyIfRequested': False,
    },
}

In [44]:
from nupic.frameworks.opf.model_factory import ModelFactory
model = ModelFactory.create(MODEL_PARAMS)
model.enableInference({'predictedField': 'consumption'})

In [45]:
data = getData()
for _ in xrange(5):
    record = dict(zip(data.getFieldNames(), data.next()))
    print "input: ", record["consumption"]
    result = model.run(record)
    print "prediction: ", result.inferences["multiStepBestPredictions"][1]

input:  5.3
prediction:  5.3
input:  5.5
prediction:  5.5
input:  5.1
prediction:  5.36
input:  5.3
prediction:  5.1
input:  5.2
prediction:  5.342


In [46]:
print result

ModelResult(	predictionNumber=4
	rawInput={'timestamp': datetime.datetime(2010, 7, 2, 1, 0), 'gym': 'Balgowlah Platinum', 'consumption': 5.2, 'address': 'Shop 67 197-215 Condamine Street Balgowlah 2093'}
	sensorInput=SensorInput(	dataRow=(5.2, 1.0)
	dataDict={'timestamp': datetime.datetime(2010, 7, 2, 1, 0), 'gym': 'Balgowlah Platinum', 'consumption': 5.2, 'address': 'Shop 67 197-215 Condamine Street Balgowlah 2093'}
	dataEncodings=[array([ 0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.], dtype=float32), array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32)]
	sequenceReset=0.0
	category=-1
)
	inferences={'multiStepPredictions': {1: {5.1: 0.0088801263517415546, 5.2: 0.010775254623541418, 5.341999999999999: 0.98034461902471692}}, 'multiStepBucketLikelihoods': {1: {1

In [47]:
print "anomaly score: ", result.inferences["anomalyScore"]

anomaly score:  0.4


__See Subutai's talk for more info on anomaly detection!__

# Built-in OPF Clients

`python examples/opf/bin/OpfRunExperiment.py examples/opf/experiments/multistep/hotgym/`

Outputs `examples/opf/experiments/multistep/hotgym/inference/DefaultTask.TemporalMultiStep.predictionLog.csv`

`python bin/run_swarm.py examples/opf/experiments/multistep/hotgym/permutations.py`

Outputs `examples/opf/experiments/multistep/hotgym/model_0/description.py`